## LLM

In [1]:
import matplotlib.pyplot as plt
import random
from IPython.display import clear_output
from src.tokenizer import TokenizerBPE, word_split, normalize_to_ascii

import os
import time
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf
import numpy as np
import pickle as pkl
from tqdm.notebook import tqdm
from src.transformer import *
from src.data_handling import read_first_n, sample_batch


print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))



Num GPUs Available:  1


In [2]:
#tf.keras.mixed_precision.set_global_policy('mixed_float16')

In [2]:
tokenizer = pkl.load(open("tokenizers/tokenizer_CNN8000_lowercase.pkl", 'rb'))
tokenizer.create_hash()

random.seed(42)
corpus_indicies = pkl.load(open('corpus/CNN_tokenized8000_lowercase.pkl', 'rb'))
random.shuffle(corpus_indicies)

In [3]:
len(corpus_indicies)

25000

## Define Model

In [4]:
class WarmUpThenDecay(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self,
                 initial_learning_rate: float,
                 warmup_steps: int,
                 decay_schedule_fn: tf.keras.optimizers.schedules.LearningRateSchedule):
        """
        initial_learning_rate: peak LR reached at end of warmup
        warmup_steps:      # of steps to ramp from 0 → initial_learning_rate
        decay_schedule_fn: a tf.keras schedule to apply *after* warmup
        """
        super().__init__()
        self.initial_lr = initial_learning_rate
        self.warmup_steps = warmup_steps
        self.decay_schedule_fn = decay_schedule_fn

    def __call__(self, step):
        # Cast to float32 for safety in graph mode
        step = tf.cast(step, tf.float32)
        warmup_steps = tf.cast(self.warmup_steps, tf.float32)

        # compute linear warmup: lr = initial_lr * (step / warmup_steps)
        warmup_lr = self.initial_lr * (step / warmup_steps)

        # after warmup_steps, switch to decay schedule (shift step count)
        decay_step = step - warmup_steps
        decay_lr = self.decay_schedule_fn(decay_step)

        # if step < warmup_steps, pick warmup_lr, else decay_lr
        return tf.cond(step < warmup_steps,
                       lambda: warmup_lr,
                       lambda: decay_lr)
    


In [ ]:
import tensorflow as tf
import numpy as np


class DenseLayer(tf.keras.Model):
    def __init__(self, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        d_xavier = np.sqrt(6.0 / (input_dim + output_dim))
        self.W = tf.Variable(tf.random.uniform([input_dim, output_dim], -1 / d_xavier , 1 / d_xavier))
        self.b = tf.Variable(tf.zeros([output_dim]))

        self.parameter_list = [self.W, self.b]

    def __call__(self, x):
        return tf.linalg.matmul(x, self.W) + self.b


class TransformerBlock(tf.keras.Model):
    def __init__(
        self,
        vocab_size,
        max_seq_len,
        heads,
        embed_dim,
        ff_dim,
        dropout,
        **kwargs

    ):
        super().__init__(**kwargs)
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.heads = heads
        self.max_seq_len = max_seq_len
        self.ff_dim = ff_dim
        self.dropout = dropout

        self.head_dim = embed_dim // heads
        self.dol1 = tf.keras.layers.Dropout(dropout)
        self.dol2 = tf.keras.layers.Dropout(dropout)
        self.dol3 = tf.keras.layers.Dropout(dropout)

        d = tf.sqrt(tf.cast(self.embed_dim, tf.float32))

        self.KQV = tf.Variable(
            tf.random.uniform([embed_dim, 3*embed_dim], -1 / d, 1 / d), name="KQV"
        )
        self.WO = tf.Variable(
            tf.random.uniform([embed_dim, embed_dim], -1 / d, 1 / d), name="WO"
        )

        self.layer_up = DenseLayer(embed_dim, ff_dim)
        self.layer_down = DenseLayer(ff_dim, embed_dim)

        self.ln1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.ln1.build((None, None, embed_dim))

        self.ln2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.ln2.build((None, None, embed_dim))

        self.parameter_list = [
            self.KQV,
            self.WO,
        ]
        self.parameter_list += self.layer_up.parameter_list
        self.parameter_list += self.layer_down.parameter_list
        self.parameter_list += self.ln1.trainable_variables
        self.parameter_list += self.ln2.trainable_variables

        self.parameter_decay = [
            self.KQV,
            self.WO,
            self.layer_up.W,
            self.layer_down.W,
        ]

    def attention(self, x_embeds, training=False):
        batch = tf.shape(x_embeds)[0]
        seq = tf.shape(x_embeds)[1]

        x_kqv = tf.matmul(x_embeds, self.KQV)
        x_kqv = tf.reshape(x_kqv, [batch, seq, self.heads, 3, self.head_dim])
        x_kqv = tf.transpose(x_kqv, [0, 2, 3, 1, 4])
        x_k = x_kqv[:, :, 0, :, :]
        x_q = x_kqv[:, :, 1, :, :]
        x_v = x_kqv[:, :, 2, :, :]


        inner = tf.einsum("bijl, bikl -> bijk", x_q, x_k)
        mask = tf.linalg.band_part(tf.ones((1, 1, seq, seq), dtype=tf.bool), -1, 0)

        inner_masked = tf.where(mask, inner, tf.constant(-np.inf))
        

        dk = tf.sqrt(tf.cast(self.head_dim, tf.float32))
        WA = tf.nn.softmax(inner_masked / dk, axis=-1)
        WA = self.dol1(WA, training)

        head_outs = WA @ x_v
        concat = tf.transpose(head_outs, [0, 2, 1, 3])  # [batch, seq, heads, head_dim]
        out = tf.reshape(concat, [batch, seq, self.embed_dim])
        out = tf.einsum("ijk,kl -> ijl", out, self.WO)

        # pre-norm to keep gradients alive
        out = self.dol2(out)
        out = self.ln1(out)
        out = out + x_embeds

        return out
    
    def ffnn(self, x_embeds, training=False):
        out = self.layer_up(x_embeds)
        out = tf.nn.gelu(out)
        out = self.layer_down(out)
        out = self.dol3(out, training)
        out = self.ln2(out)
        out = out + x_embeds

        return out
    
    def call(self, x_embeds, training=False):
        x_embeds = self.attention(x_embeds, training)
        x_embeds = self.ffnn(x_embeds, training)

        return x_embeds



class Transformer(tf.keras.Model):
    def __init__(
        self,
        vocab_size,
        max_seq_len,
        tf_blocks,
        embed_dim,
        heads,
        ff_dim,
        unembed_dims,
        lr,
        wd=None,
        dropout=0.1,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.heads = heads
        self.max_seq_len = max_seq_len
        self.tf_blocks = tf_blocks
        self.ff_dim = ff_dim
        self.unembed_dims = unembed_dims
        self.wd = wd
        self.dropout = dropout

        self.head_dim = embed_dim // heads
        self.dol = tf.keras.layers.Dropout(dropout)


        d = tf.sqrt(tf.cast(self.embed_dim, tf.float32))

        self.word_embed = tf.Variable(
            tf.random.uniform([vocab_size, embed_dim], -1 / d, 1 / d), name="W_embed"
        )
        self.pos_embed = tf.Variable(
            tf.random.uniform([max_seq_len, embed_dim], -1 / d, 1 / d), name="W_pos_embed"
        )

        self.tf_blocks = []
        for i in range(tf_blocks):
            self.tf_blocks.append(TransformerBlock(vocab_size, max_seq_len, heads, embed_dim, ff_dim, dropout))
    

        self.unembed_b = tf.Variable(tf.zeros([vocab_size]))
        self.parameter_list = [
            self.word_embed,
            self.pos_embed,
        ]

        for block in self.tf_blocks:
            self.parameter_list += block.parameter_list
        
        self.parameter_list.append(self.unembed_b)

        self.parameter_decay = [
            self.word_embed,
            self.pos_embed,]
        
        for block in self.tf_blocks:
            self.parameter_decay += block.parameter_decay

        self.optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

    def call(self, x, training=False, logits=True, ):

        x = self.embed(x, training)
        for block in self.tf_blocks:
            x = block.call(x, training)
        
        if logits:
            x = self.unembed(x)

        return x
    
    def embed(self, x, training=False):
        seq = tf.shape(x)[1]
        if seq > self.max_seq_len:
            x = x[:, -self.max_seq_len :]
            seq = self.max_seq_len
        x_embeds = tf.nn.embedding_lookup(self.word_embed, x)
        x_embeds = x_embeds + tf.expand_dims(self.pos_embed[:seq], axis=0)
        x_embeds = self.dol(x_embeds, training)

        return x_embeds

    def unembed(self, x_embeds):
        logits = x_embeds @ tf.transpose(self.word_embed) + self.unembed_b

        return logits

    @tf.function()
    def train_step(self, indices, y_true):

        with tf.GradientTape() as tape:
            loss = self.evaluate(indices, y_true, training=True)

        grads = tape.gradient(loss, self.parameter_list)
        self.optimizer.apply_gradients(zip(grads, self.parameter_list))

        get_lr = self.optimizer._decayed_lr(tf.float32)
        
        if self.wd is not None:
            for param in self.parameter_decay:
                param.assign_sub(get_lr*self.wd * param)
                
        return loss

    def evaluate(self, indices, y_true, training = False):
        y_true = y_true[:, 1:]
        
        y_pred = self.call(indices, training)[:, :-1]
        loss = tf.math.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred, from_logits=True))
        return loss

In [57]:
initial_lr = 1e-4
decay_steps = 10000
decay_rate = 0.5
decay_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_lr,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=False)

warmup_steps = 5000
lr_schedule = WarmUpThenDecay(
    initial_learning_rate=initial_lr,
    warmup_steps=warmup_steps,
    decay_schedule_fn=decay_schedule)

np.random.seed(42)
tf.random.set_seed(42)

max_seq_len = 128
embed_dim = 768
tf_blocks = 6
heads = 6
ff_dim = 4*embed_dim
weight_decay = 0.01
dropout = 0.05

unembed_dims = []

model = Transformer(vocab_size=tokenizer.vocab_size,
                    max_seq_len=max_seq_len,
                    embed_dim=embed_dim,
                    tf_blocks=tf_blocks,
                    heads=heads,
                    ff_dim = ff_dim,
                    unembed_dims=unembed_dims,
                    lr=lr_schedule,
                    wd = weight_decay,
                    dropout=dropout,
                    )

losses_train = []
losses_test = []


In [58]:
name = "model_8k_tokens_largeBatch"


ckpt = tf.train.Checkpoint(
    optimizer=model.optimizer,
    model=model
)
ckpt_manager = tf.train.CheckpointManager(
    ckpt, 
    directory="checkpoints/" + name,      # folder where ckpts are saved
    max_to_keep=5                         # only keep 5 latest checkpoints
)

In [61]:
ckpt.restore(ckpt_manager.latest_checkpoint)
losses_train, losses_test = pkl.load(open("checkpoints/losses_" + name + ".pkl", 'rb'))

In [26]:
total_params = 0
for var in model.parameter_list:
    shape = var.get_shape()
    num_params = 1
    for dim in shape:
        num_params *= dim
    total_params += num_params
print(f"Total number of parameters: {total_params}")

Total number of parameters: 48811396


In [82]:
batch_size = 32
for i in tqdm(range(100000)):
    indices, y_true = sample_batch(corpus_indicies[:20000], 
                                   batch_size, 
                                   tokenizer, 
                                   max_seq_len)
    
    
    loss_train = model.train_step(indices, y_true).numpy()
    losses_train.append(loss_train)


    indices, y_true = sample_batch(corpus_indicies[20000:], 
                                batch_size//4, 
                                tokenizer, 
                                max_seq_len)
    
    loss_test = model.evaluate(indices, y_true).numpy()
        
    losses_test.append(loss_test)

    if (i+1) % 1000 == 0:
        ckpt_manager.save()
        pkl.dump([losses_train, losses_test], open("checkpoints/losses_" + name + ".pkl", 'wb'))


    lr = model.optimizer._decayed_lr(tf.float32).numpy()
    clear_output(wait=True)

    # prepare x-axis for the last 400 steps
    start = max(0, len(losses_train) - 1000)
    x_zoom = np.arange(start, len(losses_train))

    fig, axes = plt.subplots(2, 1, figsize=(10, 8), sharex=False)

    # Top subplot: zoom on last 400 steps
    ax1 = axes[0]
    ax1.plot(x_zoom, losses_test[-1000:], label="Test Loss")
    ax1.plot(x_zoom, losses_train[-1000:], label="Train Loss")

    _min = min(losses_train[-1000:] + losses_test[-1000:])
    _max = max(losses_train[-1000:] + losses_test[-1000:])
    delta = _max - _min
    #ax1.set_ylim(_min - 0.1 * delta, _max + 0.1 * delta)

    ax1.set_title("Training Loss (Last 1000 Steps)")
    ax1.set_xlabel("Step")
    ax1.set_ylabel("Loss")
    ax1.legend()
    ax1.grid(True)

    # Bottom subplot: full series
    ax2 = axes[1]
    ax2.plot(losses_test[500:], label="Test Loss")
    ax2.plot(losses_train[500:], label="Train Loss, lr = {:.2e}".format(lr))

    ax2.set_title("Training Loss (Full Series)")
    ax2.set_xlabel("Step")
    ax2.set_ylabel("Loss")
    ax2.legend()
    ax2.grid(True)

    plt.tight_layout()
    plt.show()

KeyboardInterrupt: 

In [83]:
def tokenize(indices, merge_list):
    indices = np.array(indices)
    for pair, new_idx in merge_list:
        slice = np.where(np.logical_and(indices[:-1] == pair[0],  indices[1:] == pair[1]))
        if len(slice[0]) > 0:
            indices[:-1][slice] = new_idx
            indices = np.delete(indices, (slice[0]+1))

    return tf.expand_dims(tf.convert_to_tensor(indices, dtype=tf.int32), axis=0)

In [90]:
text = "obama's"
text = text.lower()

indices = tf.cast(tokenizer.tokenizer.tokenize(text), tf.int32)
indices = tokenize(indices, tokenizer.merge_list)
print(indices)

tf.Tensor([[2318]], shape=(1, 1), dtype=int32)


In [92]:
T = 0.5
tf.random.set_seed(43)
for i in range(128):
    logits = model.call(indices)[0,-1:]
    idx = tf.cast(tf.random.categorical(logits/T, num_samples=1), tf.int32)
    indices = tf.concat([indices, idx], axis=1)
    text_pred = tokenizer.detokenize(indices)
    text_pred = text_pred.numpy()[0].decode('utf-8').replace("\n", " ")
    print(text_pred, end='\r', flush=True)
    #time.sleep(0.05)


In [93]:
from sklearn.cluster import KMeans

def cosine_similarity(embed_a, embed_b):
    """
    Compute the cosine similarity between two vectors.
    """
    embed_b_T = tf.transpose(embed_b)
    dot_product = embed_a@embed_b_T
    
    norm_a = tf.linalg.norm(embed_a, axis=1, keepdims=True)
    norm_b = tf.linalg.norm(embed_b_T, axis=0, keepdims=True)

    return dot_product / (norm_a * norm_b)


def cluster(X, n_clusters):
    X = X/np.linalg.norm(X, axis=1, keepdims=True)

    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(X)
    inertia = kmeans.inertia_
    labels = kmeans.labels_
    clusters = kmeans.cluster_centers_

    return inertia, labels, clusters


class EmbeddingClustering:
    def __init__(self, tokenizer, n_clusters=10):
        
        self.tokenizer = tokenizer
        self.n_clusters = n_clusters

    def fit(self, word_embed):
        inertia, labels, clusters = cluster(word_embed, self.n_clusters)
        self.word_embed = word_embed
        self.inertia = inertia
        self.labels = labels
        self.clusters = tf.convert_to_tensor(clusters, dtype=tf.float32)

        cos_sim = cosine_similarity(self.clusters, word_embed)
        self.idx_list =  tf.argsort(cos_sim, axis=-1, direction='DESCENDING', stable=False, name=None)

    def print_clusters(self, n_words=10):
        for idx in self.idx_list:
            for i in idx[:n_words]:
                word = self.tokenizer.detokenize(tf.expand_dims(tf.cast(i, tf.int32), axis=0))
                word = word.numpy().decode('utf-8')
                print(word)
            print("\n")

In [94]:
word_embed = model.word_embed
embedding_clustering = EmbeddingClustering(tokenizer, n_clusters=100)
embedding_clustering.fit(word_embed)
embedding_clustering.print_clusters(n_words=10)

punish
restrain
assum
accomplish
interven
disagree
commit
adjust
embarrass
proceed


born
connected
speaking
packed
talking
traveling
gathered
working
deployed
struggled


legislative
legal
federal
criminal
formal
constitutional
current
judicial
supreme
professional


oklahom
minnes
matthe
princi
trium
thomp
cele
wim
franch
unte


iet
ork
inois
aste
ool
aff
helicop
uck
ld
igh


vehicles
homes
buildings
stations
airports
boats
towns
equipment
spots
animals


absolutely
really
pretty
always
simply
probably
totally
usually
rarely
essentially


liverpool
midfielder
portugal
england
argentina
barcelona
goalkeeper
soccer
ferrari
striker


genu
^
includ
lished
bollah
apore
estab
massachu
issip
inclu


tun
hurrican
nutr
kne
exhib
var
gg
conven
hay
nam


republicans
governments
observers
leaders
lawmakers
conservatives
candidates
analysts
lawyers
critics


republican
conservative
democratic
hispanic
liberal
communist
former
independent
elected
senior


quiet
beautiful
perfect
genuine
unique
won

# Overlap

In [112]:
word_embed = model.word_embed

text = "queen"
text = text.lower()

idx = tf.cast(tokenizer.tokenizer.tokenize(text), tf.int32)
idx = tokenize(idx, tokenizer.merge_list)
print(idx)
embed1 = tf.expand_dims(word_embed[idx[0][0]], axis=0)


text = "woman"
text = text.lower()

idx = tf.cast(tokenizer.tokenizer.tokenize(text), tf.int32)
idx = tokenize(idx, tokenizer.merge_list)
print(idx)
embed2 = tf.expand_dims(word_embed[idx[0][0]], axis=0)

text = "man"
text = text.lower()

idx = tf.cast(tokenizer.tokenizer.tokenize(text), tf.int32)
idx = tokenize(idx, tokenizer.merge_list)
print(idx)
embed3 = tf.expand_dims(word_embed[idx[0][0]], axis=0)

embed = embed1/tf.norm(embed1) - embed2/tf.norm(embed2) + embed3/tf.norm(embed3)

cosine_sim = embed@tf.transpose(word_embed)
idx = tf.argsort(cosine_sim, axis=-1, 
                 direction='DESCENDING',
                 #direction='ASCENDING', 
                 stable=False, name=None)[0]

for i in idx[:100]:
    i = tf.expand_dims(i, axis=0)
    print(tokenizer.detokenize(i).numpy().decode('utf-8'))

tf.Tensor([[4225]], shape=(1, 1), dtype=int32)
tf.Tensor([[964]], shape=(1, 1), dtype=int32)
tf.Tensor([[187]], shape=(1, 1), dtype=int32)
queen
man
richard
mar
nic
prince
des
ca
van
friend
la
yan
pri
mon
fro
hop
ju
wife
lie
dest
epis
te
star
pa
bra
hi
daniel
bi
mic
jane
p
simp
ar
nor
den
adm
hua
198
lu
fa
miami
ham
bur
u
stron
fol
stat
clark
george
m
ra
legendary
ch
ja
phe
sla
stone
cy
hoff
bron
sha
year's
canad
cop
son
d
cape
hu
glas
isa
smu
rev
dy
(
vo
tan
mu
louis
pic
ti
pra
former
ta
da
bureau
aven
tom
l
rw
de
lou
mary
an
mum
kes
li
leon
photograp
dem
christ


## 